In [26]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
train = pd.read_csv('../input/train.csv', usecols = [1,2,3,4,5],
                   dtype = {'onpromotion': bool},parse_dates = ['date'],
                    skiprows = range(1, 66458909))

In [5]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,12.0,False
1,2016-01-01,25,105575,9.0,False
2,2016-01-01,25,105857,3.0,False
3,2016-01-01,25,108634,3.0,False
4,2016-01-01,25,108701,2.0,True


In [6]:
test = pd.read_csv('../input/test.csv', usecols = [0,1,2,3,4],
                  dtype = {'onpromotion': bool}, parse_dates = ['date']).set_index(['store_nbr','item_nbr','date'])

In [7]:
test.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [8]:
train['unit_sales'] = train['unit_sales'].apply(lambda x: np.log1p(float(x)) if float(x)>0 else 0)

In [9]:
items = pd.read_csv('../input/items.csv').set_index('item_nbr')
df_2017 = train[train['date']>date(2017,1,1)]

In [10]:
promo_2017_train = df_2017.set_index(['store_nbr','item_nbr','date'])
print (promo_2017_train.head())
print (promo_2017_train.shape)

                               unit_sales  onpromotion
store_nbr item_nbr date                               
1         103520   2017-01-02    0.693147        False
          105575   2017-01-02    1.386294        False
          105577   2017-01-02    0.693147        False
          105737   2017-01-02    0.693147        False
          108079   2017-01-02    1.098612        False
(23806568, 2)


In [11]:
promo_2017_train = promo_2017_train[['onpromotion']].unstack(level = -1).fillna(False)
#singel bracket return a series, double brackets return a dataframe
#stack method turns column names into index values, and the unstack method turns index values into column names
promo_2017_train.shape

(167515, 226)

In [12]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_train.head()

date                2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-06  2017-01-07  2017-01-08  2017-01-09  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574          True       False       False       False   

date                2017-01-10  2017-01-11     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 226 columns]

In [13]:
promo_2017_test = test[['onpromotion']].unstack(level = -1).fillna(False)

In [14]:
promo_2017_test.head(3)

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False

In [15]:
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)

In [20]:
promo_2017_test.sample(3)

,date,2017-08-16 00:00:00,2017-08-17 00:00:00,2017-08-18 00:00:00,2017-08-19 00:00:00,2017-08-20 00:00:00,2017-08-21 00:00:00,2017-08-22 00:00:00,2017-08-23 00:00:00,2017-08-24 00:00:00,2017-08-25 00:00:00,2017-08-26 00:00:00,2017-08-27 00:00:00,2017-08-28 00:00:00,2017-08-29 00:00:00,2017-08-30 00:00:00,2017-08-31 00:00:00
store_nbr,item_nbr,,,,,,,,,,,,,,,,
40,1660191,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
29,2045986,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
28,2048163,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [21]:
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

In [24]:
promo_2017 = pd.concat([promo_2017_train,promo_2017_test], axis = 1)

In [27]:
promo_2017.tail(3)

date                2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-06  2017-01-07  2017-01-08  2017-01-09  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-10  2017-01-11  2017-01-12  2017-01-13  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-14  2017-01-15  2017-01-16  2017-01-17  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-18  2017-01-19  2017-01-20  2017-01-21  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-22  2017-01-23  2017-01-24  2017-01-25  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-26  2017-01-27  2017-01-28  2017-01-29  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-30  2017-01-31  2017-02-01  2017-02-02  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-02-03  2017-02-04  2017-02-05  2017-02-06  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-02-07  2017-02-08  2017-02-09  2017-02-10  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-02-11  2017-02-12  2017-02-13  2017-02-14  \
store_nbr item_nbr                                                   
54        2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-02-15  2017-02-16  2017-02-17  2017-02-18  \
store_nbr item_nbr                                                   

In [28]:
del promo_2017_test, promo_2017_train

In [29]:
df_2017 = df_2017.set_index(['store_nbr', 'item_nbr', 'date'])[['unit_sales']].unstack(level = -1).fillna(0)

In [31]:
df_2017.shape

(167515, 226)

In [32]:
df_2017.sample(3)

unit_sales                                              \
date               2017-01-02 2017-01-03 2017-01-04 2017-01-05 2017-01-06   
store_nbr item_nbr                                                          
29        1975649    1.609438   1.609438   0.693147   1.609438   1.098612   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   1.098612   1.386294   1.098612   1.386294   

                                                                           \
date               2017-01-07 2017-01-08 2017-01-09 2017-01-10 2017-01-11   
store_nbr item_nbr                                                          
29        1975649    1.945910   1.791759   1.609438   1.791759   1.609438   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    1.386294   1.098612   0.693147   0.000000   1.098612   

                                                                           \
date               2017-01-12 2017-01-13 2017-01-14 2017-01-15 2017-01-16   
store_nbr item_nbr                                                          
29        1975649    2.079442   0.693147   1.791759   2.302585   1.791759   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   1.609438   0.693147   1.386294   1.098612   

                                                                           \
date               2017-01-17 2017-01-18 2017-01-19 2017-01-20 2017-01-21   
store_nbr item_nbr                                                          
29        1975649    2.079442   1.609438   1.386294   1.945910   2.197225   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    1.098612   1.386294   0.693147   1.386294   1.386294   

                                                                           \
date               2017-01-22 2017-01-23 2017-01-24 2017-01-25 2017-01-26   
store_nbr item_nbr                                                          
29        1975649    1.609438   1.386294   0.693147   1.386294   1.098612   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   1.386294   1.945910   1.386294   0.000000   

                                                                           \
date               2017-01-27 2017-01-28 2017-01-29 2017-01-30 2017-01-31   
store_nbr item_nbr                                                          
29        1975649    1.791759   1.791759   1.609438   1.386294   1.791759   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   0.000000   0.000000   0.000000   1.098612   

                                                                           \
date               2017-02-01 2017-02-02 2017-02-03 2017-02-04 2017-02-05   
store_nbr item_nbr                                                          
29        1975649    1.791759   1.386294   0.693147   1.386294   2.197225   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   0.000000   0.693147   0.000000   0.693147   

                                                                           \
date               2017-02-06 2017-02-07 2017-02-08 2017-02-09 2017-02-10   
store_nbr item_nbr                                                          
29        1975649    1.386294   2.079442   1.386294   1.386294   0.693147   
53        523054     0.000000   0.000000   0.000000   0.000000   0.000000   
32        1930967    0.000000   0.000000   1.609438   2.197225   1.609438   

                                                                           \
date               2017-02-11 2017-02-12 2017-02-13 2017-02-14 2017-02-15   
store_nbr item_nbr                                                          
29        1975649    1.609438   1.609438   1.098612   1.609438   1.098612   
53     

In [41]:
df_2017.columns.get_level_values(1)

DatetimeIndex(['2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09',
               '2017-01-10', '2017-01-11',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=226, freq=None)

In [42]:
df_2017.columns = df_2017.columns.get_level_values(1)
items = items.reindex(df_2017.index.get_level_values(1))

In [43]:
df_2017.tail()

date                2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-06  2017-01-07  2017-01-08  2017-01-09  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-10  2017-01-11  2017-01-12  2017-01-13  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-14  2017-01-15  2017-01-16  2017-01-17  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-18  2017-01-19  2017-01-20  2017-01-21  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-22  2017-01-23  2017-01-24  2017-01-25  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-26  2017-01-27  2017-01-28  2017-01-29  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-01-30  2017-01-31  2017-02-01  2017-02-02  \
store_nbr item_nbr                                                   
54        2109909          0.0         0.0         0.0         0.0   
          2110456          0.0         0.0         0.0         0.0   
          2113343          0.0         0.0         0.0         0.0   
          2113914          0.0         0.0         0.0         0.0   
          2116416          0.0         0.0         0.0         0.0   

date                2017-02-03  2017-02-04  2017-02-05  2017-02-06  \
st

In [53]:
df_2017.shape

(167515, 226)

In [44]:
def get_timespan(df,dt,minus,periods, freq = 'D'):
    return df[
        pd.date_range(dt-timedelta(days = minus), periods = periods, freq = freq)
    ]
# pd.date_range, of the three parameters: start, end, periods, exactly two must be specified

In [72]:
eg_dt = date(2017, 5,31)
get_timespan(df_2017, eg_dt, 3, 10)

date                2017-05-28  2017-05-29  2017-05-30  2017-05-31  \
store_nbr item_nbr                                                   
1         96995       0.693147    0.000000    0.000000    0.693147   
          99197       1.098612    0.693147    0.000000    0.693147   
          103520      0.000000    0.000000    0.000000    1.386294   
          103665      0.000000    1.098612    0.000000    2.197225   
          105574      0.693147    1.386294    1.098612    1.386294   
          105575      1.386294    1.945910    2.302585    1.791759   
          105577      0.000000    1.386294    0.000000    1.609438   
          105693      0.000000    0.000000    0.000000    0.000000   
          105737      1.609438    1.098612    0.693147    1.098612   
          105857      1.386294    1.945910    1.791759    2.197225   
          106716      0.693147    0.693147    1.609438    1.609438   
          108079      0.000000    0.000000    0.000000    0.693147   
          108634      0.000000    0.000000    0.693147    0.000000   
          108696      0.693147    1.791759    1.386294    1.791759   
          108698      1.609438    1.945910    1.098612    1.609438   
          108701      0.693147    0.693147    0.693147    1.386294   
          108786      1.098612    0.693147    0.693147    1.609438   
          108797      1.386294    1.098612    1.098612    0.693147   
          108831      0.000000    0.000000    0.000000    0.000000   
          108862      0.000000    0.000000    0.000000    0.000000   
          108952      0.693147    0.693147    1.945910    0.693147   
          111223      1.386294    1.791759    2.397895    2.484907   
          111397      0.693147    1.609438    0.000000    0.693147   
          112830      0.000000    1.098612    0.693147    1.098612   
          114778      1.609438    1.386294    1.386294    0.693147   
          114790      1.098612    1.945910    1.386294    1.791759   
          114799      0.000000    0.693147    0.000000    0.000000   
          114800      1.098612    0.693147    0.693147    1.791759   
          115267      0.693147    0.693147    1.098612    0.000000   
          115611      0.693147    2.708050    2.302585    2.079442   
...                        ...         ...         ...         ...   
54        2081056     1.791759    0.693147    1.609438    1.609438   
          2081064     4.709530    4.356709    4.025352    4.553877   
          2081095     1.945910    1.609438    1.791759    1.386294   
          2081142     0.000000    0.000000    0.000000    1.609438   
          2081161     4.077537    3.784190    3.610918    2.890372   
          2083837     0.000000    0.000000    0.000000    1.945910   
          2083859     0.000000    1.609438    0.000000    1.386294   
          2084278     0.000000    2.079442    0.000000    0.000000   
          2084557     2.890372    1.945910    1.791759    2.079442   
          2086882     1.609438    0.000000    0.693147    1.098612   
          2087374     0.000000    0.000000    0.000000    0.000000   
          2087409     0.000000    0.000000    0.000000    0.000000   
          2087543     0.000000    0.000000    0.000000    0.000000   
          2087567     0.000000    0.000000    0.000000    0.000000   
          2087933     0.000000    0.000000    0.000000    0.000000   
          2087978     1.609438    0.000000    1.791759    1.791759   
          2088922     2.564949    2.484907    2.772589    2.484907   
          2089036     0.000000    0.000000    0.000000    0.000000   
          2089339     2.079442    1.386294    1.609438    2.302585   
          2099230     1.098612    0.000000    0.000000    0.000000   
          2101795     0.000000    0.000000    0.000000    0.000000   
          2103250     1.386294    0.693147    0.000000    0.000000   
          2105347     1.386294    0.000000    1.386294    0.693147   
          2106464     0.000000    0.000000    0.000000    0.000000   
          

In [45]:
d = timedelta(days = 0.1)
d.seconds
# the days to seconds translation can only happen when days < 1. 
#0 <= microseconds < 1000000
#0 <= seconds < 3600 * 24 (number of seconds in one day)
#-000000000 <= days <= 999999999
date(2017, 8,16) - date(2017,7,26)


datetime.timedelta(21)

In [47]:
print (df_2017.head())

date                2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.000000    1.386294    0.693147    0.693147   
          103520      0.693147    1.098612    0.000000    1.098612   
          103665      0.000000    0.000000    1.386294    1.098612   
          105574      0.000000    1.791759    2.564949    2.302585   

date                2017-01-06  2017-01-07  2017-01-08  2017-01-09  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    1.098612    0.000000    0.000000   
          103520      1.386294    0.693147    0.000000    0.693147   
          103665      1.098612    0.693147    1.098612    0.000000   
          105574      1.945910    1.609438    1.098612    1.386294   

date              

In [48]:
def prepare_dataset(t2017, is_train = True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range (16):
        X['promo={}'.format(i)] = promo_2017[
            t2017 + timedelta(days = i)].values.astype(np.uint8)
    if is_train:
        y =df_2017[
            pd.date_range (t2017, periods = 16)
        ].values
        return X, y
    return X
    

In [49]:
print ('preparing dataset ...')
t2017 = date(2017, 5, 31)
test_X, test_y = prepare_dataset(t2017+ timedelta(days = 7))
test_X.head()

preparing dataset ...


,day_1_2017,mean_140_2017,mean_14_2017,mean_30_2017,mean_3_2017,mean_60_2017,mean_7_2017,promo_140_2017,promo_14_2017,promo_60_2017,mean_4_dow0_2017,mean_20_dow0_2017,mean_4_dow1_2017,mean_20_dow1_2017,mean_4_dow2_2017,mean_20_dow2_2017,mean_4_dow3_2017,mean_20_dow3_2017,mean_4_dow4_2017,mean_20_dow4_2017,mean_4_dow5_2017,mean_20_dow5_2017,mean_4_dow6_2017,mean_20_dow6_2017,promo=0,promo=1,promo=2,promo=3,promo=4,promo=5,promo=6,promo=7,promo=8,promo=9,promo=10,promo=11,promo=12,promo=13,promo=14,promo=15
0,0.000000,0.075107,0.198042,0.161734,0.000000,0.156940,0.099021,0,0,0,0.173287,0.069315,0.173287,0.034657,0.173287,0.179176,0.173287,0.103972,0.173287,0.034657,0.000000,0.069315,0.000000,0.034657,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.174485,0.828418,0.625496,0.732408,0.312748,1.045884,0,0,0,0.895880,0.179176,1.039721,0.317805,0.722593,0.144519,0.486478,0.201268,0.549306,0.179176,0.722593,0.144519,0.274653,0.054931,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.609438,0.738711,0.881969,0.828724,0.767528,0.854054,0.939893,0,0,0,0.922220,0.926689,1.242453,1.008501,0.934417,0.813378,1.069167,0.692582,0.000000,0.158903,0.519860,0.754097,1.079372,0.816828,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.791759,1.049640,1.073373,1.095595,1.425555,1.025186,1.436773,0,0,0,0.549306,1.029874,0.794513,1.003711,1.572892,1.393903,1.660947,1.502511,0.447940,0.508139,1.170533,0.865082,1.343820,1.044258,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2.079442,1.816292,1.551652,1.750672,1.617343,1.753564,1.700184,34,0,14,2.011397,2.106416,1.818100,2.077306,1.989294,1.934902,1.487661,1.626346,0.794513,0.909643,1.779099,2.031329,2.036967,2.028106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
test_X.shape

(167515, 40)

In [50]:
t2017 = date(2017, 5, 31)
X_1, y_1 = [],[]
for i in range (6):
    delta = timedelta(days = 7*i)
    X_tmp, y_tmp = prepare_dataset(
        t2017+delta
    )
    X_1.append(X_tmp)
    y_1.append(y_tmp)
X_train = pd.concat(X_1, axis = 0)
y_train = np.concatenate(y_1, axis = 0)
del X_1, y_1
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train = False)

In [51]:
print (X_test.head())

   day_1_2017  mean_140_2017  mean_14_2017  mean_30_2017  mean_3_2017  \
0    0.000000       0.153952      0.334438      0.275522     0.000000   
1    0.000000       0.376532      0.206455      0.331321     0.000000   
2    0.000000       0.821010      0.573577      0.714515     0.231049   
3    0.693147       1.040541      1.031388      1.017638     0.462098   
4    1.609438       1.765433      1.629185      1.714960     0.998577   

   mean_60_2017  mean_7_2017  promo_140_2017  promo_14_2017  promo_60_2017  \
0      0.160866     0.099021               0              0              0   
1      0.444620     0.156945               0              0              0   
2      0.756274     0.495105               0              0              0   
3      1.001216     0.980990               0              0              0   
4      1.686812     1.560437              24              0              0   

   mean_4_dow0_2017  mean_20_dow0_2017  mean_4_dow1_2017  mean_20_dow1_2017  \
0          0.

In [199]:
import time
start = time.time()
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 8
}
#for the num_threads, set it to the real CPU cores, not the number of threads
#device, choose device for the tree learning, you can use GPU to achieve the 
#faster learning
#device, default = cpu, options = cpu, gpu
MAX_ROUNDS = 1000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))
end = time.time()

Step 1


c:\users\rula\python\lib\site-packages\lightgbm\basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.319638	valid_1's l2: 0.308008
[100]	training's l2: 0.305873	valid_1's l2: 0.296643
[150]	training's l2: 0.302827	valid_1's l2: 0.294481
[200]	training's l2: 0.301147	valid_1's l2: 0.293674
[250]	training's l2: 0.299879	valid_1's l2: 0.293148
[300]	training's l2: 0.298871	valid_1's l2: 0.292836
[350]	training's l2: 0.297914	valid_1's l2: 0.29257
[400]	training's l2: 0.297085	valid_1's l2: 0.292374
[450]	training's l2: 0.296312	valid_1's l2: 0.29222
[500]	training's l2: 0.295635	valid_1's l2: 0.292119
[550]	training's l2: 0.294943	valid_1's l2: 0.292041
[600]	training's l2: 0.294305	valid_1's l2: 0.291953
[650]	training's l2: 0.29366	valid_1's l2: 0.291879
[700]	training's l2: 0.293059	valid_1's l2: 0.291814
[750]	training's l2: 0.292478	valid_1's l2: 0.291753
[800]	training's l2: 0.291905	valid_1's l2: 0.291677
[850]	training's l2: 0.291335	valid_1's l2: 0.291612
[900]	training's l2: 0.290804	valid_1's l

[50]	training's l2: 0.385659	valid_1's l2: 0.375329
[100]	training's l2: 0.369265	valid_1's l2: 0.361723
[150]	training's l2: 0.364918	valid_1's l2: 0.358702
[200]	training's l2: 0.362604	valid_1's l2: 0.357743
[250]	training's l2: 0.360793	valid_1's l2: 0.357129
[300]	training's l2: 0.359279	valid_1's l2: 0.356685
[350]	training's l2: 0.357988	valid_1's l2: 0.356348
[400]	training's l2: 0.356751	valid_1's l2: 0.356074
[450]	training's l2: 0.355553	valid_1's l2: 0.355845
[500]	training's l2: 0.354542	valid_1's l2: 0.355639
[550]	training's l2: 0.353561	valid_1's l2: 0.355559
[600]	training's l2: 0.352569	valid_1's l2: 0.35551
[650]	training's l2: 0.351668	valid_1's l2: 0.355432
[700]	training's l2: 0.350854	valid_1's l2: 0.355386
[750]	training's l2: 0.350027	valid_1's l2: 0.355338
[800]	training's l2: 0.349216	valid_1's l2: 0.355233
[850]	training's l2: 0.348463	valid_1's l2: 0.355199
[900]	training's l2: 0.34772	valid_1's l2: 0.355174
[950]	training's l2: 0.347009	valid_1's l2: 0.355

Step 10
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.366958	valid_1's l2: 0.387241
[100]	training's l2: 0.351837	valid_1's l2: 0.372319
[150]	training's l2: 0.347752	valid_1's l2: 0.370069
[200]	training's l2: 0.345481	valid_1's l2: 0.369351
[250]	training's l2: 0.343893	valid_1's l2: 0.368961
[300]	training's l2: 0.34253	valid_1's l2: 0.368619
[350]	training's l2: 0.341323	valid_1's l2: 0.368323
[400]	training's l2: 0.340319	valid_1's l2: 0.368128
[450]	training's l2: 0.33934	valid_1's l2: 0.367982
[500]	training's l2: 0.338388	valid_1's l2: 0.367911
[550]	training's l2: 0.337487	valid_1's l2: 0.367827
[600]	training's l2: 0.336647	valid_1's l2: 0.367776
[650]	training's l2: 0.335838	valid_1's l2: 0.36775
[700]	training's l2: 0.335057	valid_1's l2: 0.367657
[750]	training's l2: 0.334287	valid_1's l2: 0.367652
[800]	training's l2: 0.333571	valid_1's l2: 0.367576
[850]	training's l2: 0.332776	valid_1's l2: 0.367469
[900]	training's l2: 0.33207	vali

[300]	training's l2: 0.356454	valid_1's l2: 0.361994
[350]	training's l2: 0.355318	valid_1's l2: 0.361815
[400]	training's l2: 0.354215	valid_1's l2: 0.361702
[450]	training's l2: 0.353247	valid_1's l2: 0.361602
[500]	training's l2: 0.352268	valid_1's l2: 0.361559
[550]	training's l2: 0.351427	valid_1's l2: 0.361557
[600]	training's l2: 0.350538	valid_1's l2: 0.361601
Early stopping, best iteration is:
[565]	training's l2: 0.351173	valid_1's l2: 0.361538
mean_30_2017: 2633927.57
mean_14_2017: 1058071.23
mean_7_2017: 550218.03
mean_60_2017: 475118.27
mean_20_dow6_2017: 380520.62
promo=13: 276567.70
mean_3_2017: 167689.79
mean_4_dow6_2017: 132575.17
day_1_2017: 41397.62
promo_14_2017: 31642.88
mean_140_2017: 27856.28
promo=14: 21647.83
promo=12: 18372.46
promo=10: 16763.70
mean_4_dow5_2017: 14844.34
promo_60_2017: 14814.63
mean_20_dow1_2017: 14380.06
promo_140_2017: 13554.71
mean_20_dow5_2017: 13460.01
mean_20_dow0_2017: 10390.36
promo=6: 10095.62
mean_4_dow1_2017: 7262.60
mean_20_dow3_2

In [ ]:
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index = df_2017.index,
    columns = pd.date_range ('2017-08-16', periods = 16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(['store_nbr','item_nbr','date'], inplace = True)


In [210]:
date(2017,7,26) - date(2017, 5,24)

datetime.timedelta(63)

In [224]:
t2017 = date(2017, 5, 24)
X_1, y_1 = [],[]
for i in range (7):
    delta = timedelta(days = 7*i)
    X_tmp, y_tmp = prepare_dataset(
        t2017+delta
    )
    X_1.append(X_tmp)
    y_1.append(y_tmp)
X_train = pd.concat(X_1, axis = 0)
y_train = np.concatenate(y_1, axis = 0)
del X_1, y_1
X_val, y_val = prepare_dataset(date(2017, 7, 19))
X_test = prepare_dataset(date(2017, 8, 16), is_train = False)

In [225]:
start = time.time()
params2 = {
    'num_leaves': 90,
    'objective': 'regression_l2',
    'max_depth': 20,
    'min_data_in_leaf': 50,
    'learning_rate': 0.5,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 8
}
MAX_ROUNDS = 1200
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 7) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))
end = time.time()

Step 1


c:\users\rula\python\lib\site-packages\lightgbm\basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.318637	valid_1's l2: 0.311416
[100]	training's l2: 0.305022	valid_1's l2: 0.298915
[150]	training's l2: 0.301948	valid_1's l2: 0.296631
[200]	training's l2: 0.300333	valid_1's l2: 0.295636
[250]	training's l2: 0.29915	valid_1's l2: 0.295067
[300]	training's l2: 0.298198	valid_1's l2: 0.294677
[350]	training's l2: 0.297315	valid_1's l2: 0.294309
[400]	training's l2: 0.296578	valid_1's l2: 0.294054
[450]	training's l2: 0.295892	valid_1's l2: 0.293857
[500]	training's l2: 0.295208	valid_1's l2: 0.293709
[550]	training's l2: 0.294594	valid_1's l2: 0.293596
[600]	training's l2: 0.293976	valid_1's l2: 0.29347
[650]	training's l2: 0.293393	valid_1's l2: 0.293369
[700]	training's l2: 0.292841	valid_1's l2: 0.29331
[750]	training's l2: 0.292284	valid_1's l2: 0.293294
[800]	training's l2: 0.291752	valid_1's l2: 0.293203
[850]	training's l2: 0.291266	valid_1's l2: 0.293142
[900]	training's l2: 0.290768	valid_1's l

Step 5
Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.383649	valid_1's l2: 0.36164
[100]	training's l2: 0.36749	valid_1's l2: 0.352586
[150]	training's l2: 0.36335	valid_1's l2: 0.350922
[200]	training's l2: 0.361234	valid_1's l2: 0.35027
[250]	training's l2: 0.359587	valid_1's l2: 0.349871
[300]	training's l2: 0.358144	valid_1's l2: 0.349585
[350]	training's l2: 0.35689	valid_1's l2: 0.349382
[400]	training's l2: 0.355764	valid_1's l2: 0.349236
[450]	training's l2: 0.354812	valid_1's l2: 0.349126
[500]	training's l2: 0.353817	valid_1's l2: 0.349001
[550]	training's l2: 0.352892	valid_1's l2: 0.348851
[600]	training's l2: 0.352043	valid_1's l2: 0.348864
[650]	training's l2: 0.351173	valid_1's l2: 0.348801
[700]	training's l2: 0.350383	valid_1's l2: 0.348746
[750]	training's l2: 0.34966	valid_1's l2: 0.348689
[800]	training's l2: 0.348966	valid_1's l2: 0.348706
Early stopping, best iteration is:
[756]	training's l2: 0.349577	valid_1's l2: 0.348677
me

[700]	training's l2: 0.340423	valid_1's l2: 0.34934
[750]	training's l2: 0.33977	valid_1's l2: 0.349223
[800]	training's l2: 0.339131	valid_1's l2: 0.349135
[850]	training's l2: 0.338513	valid_1's l2: 0.349143
Early stopping, best iteration is:
[801]	training's l2: 0.339117	valid_1's l2: 0.349132
mean_30_2017: 2771570.10
mean_14_2017: 1489207.51
mean_7_2017: 945501.52
mean_60_2017: 416261.56
mean_20_dow1_2017: 214369.90
promo=8: 198764.63
promo_14_2017: 37741.02
mean_4_dow1_2017: 35858.71
day_1_2017: 32647.65
mean_3_2017: 27431.00
promo_60_2017: 21387.77
promo=7: 20738.55
mean_20_dow2_2017: 18321.86
promo=10: 16098.19
mean_140_2017: 15760.49
promo_140_2017: 14083.51
mean_4_dow6_2017: 11811.09
mean_20_dow4_2017: 11001.59
mean_4_dow0_2017: 9430.70
mean_4_dow5_2017: 8978.16
mean_20_dow0_2017: 8253.79
mean_4_dow2_2017: 7224.09
mean_4_dow3_2017: 6807.06
mean_4_dow4_2017: 6581.45
promo=9: 6416.76
mean_20_dow6_2017: 6361.25
mean_20_dow5_2017: 5687.04
mean_20_dow3_2017: 5414.17
promo=11: 5238.

[200]	training's l2: 0.374597	valid_1's l2: 0.382162
[250]	training's l2: 0.372981	valid_1's l2: 0.381583
[300]	training's l2: 0.371662	valid_1's l2: 0.381276
[350]	training's l2: 0.370487	valid_1's l2: 0.381001
[400]	training's l2: 0.369435	valid_1's l2: 0.380801
[450]	training's l2: 0.368403	valid_1's l2: 0.380616
[500]	training's l2: 0.367511	valid_1's l2: 0.380453
[550]	training's l2: 0.366657	valid_1's l2: 0.380386
[600]	training's l2: 0.365859	valid_1's l2: 0.380338
[650]	training's l2: 0.36503	valid_1's l2: 0.380162
[700]	training's l2: 0.364242	valid_1's l2: 0.380126
[750]	training's l2: 0.363526	valid_1's l2: 0.380089
[800]	training's l2: 0.362787	valid_1's l2: 0.380045
[850]	training's l2: 0.362127	valid_1's l2: 0.380049
[900]	training's l2: 0.36142	valid_1's l2: 0.38001
Early stopping, best iteration is:
[874]	training's l2: 0.361766	valid_1's l2: 0.379983
mean_30_2017: 3651939.33
mean_14_2017: 1095655.54
mean_7_2017: 648345.80
mean_60_2017: 641659.75
mean_3_2017: 329677.83


In [226]:
print ('the total tunning time is {}'.format(end - start))

the total tunning time is 23251.97415447235


In [227]:
y_test2 = np.array(test_pred).transpose()
df_preds2 = pd.DataFrame(
    y_test2, index = df_2017.index,
    columns = pd.date_range ('2017-08-16', periods = 16)
).stack().to_frame("unit_sales")
df_preds2.index.set_names(['store_nbr','item_nbr','date'], inplace = True)


In [228]:
df_preds.head()

unit_sales
store_nbr item_nbr date                  
1         96995    2017-08-16    0.184831
                   2017-08-17    0.189326
                   2017-08-18    0.217176
                   2017-08-19    0.246337
                   2017-08-20    0.250118

In [229]:
df_preds2.head()

unit_sales
store_nbr item_nbr date                  
1         96995    2017-08-16    0.175023
                   2017-08-17    0.184139
                   2017-08-18    0.218286
                   2017-08-19    0.267237
                   2017-08-20    0.261767

In [230]:
test.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [248]:
submission = test[['id']].join(df_preds, how = 'left').fillna(0)
submission2 = test[['id']].join(df_preds2, how = 'left').fillna(0)
final_preds= 0.5*df_preds + 0.5*df_preds2
sub = test[['id']].join(final_preds, how = 'left').fillna(0)
#DataFrame.join("how") left: use calling frame's index, 
                    # right: use other frames's index,
                    # outer: from union of callling fram's index with other frame's index
                    # inner: 
submission['unit_sales'] =np.clip(np.expm1(submission['unit_sales']),0,900)
submission2['unit_sales'] =np.clip(np.expm1(submission2['unit_sales']),0,900)
# np.clip() given an interval, values outside the interval are clipped to the inteval edges. 
#submission2.to_csv('lgb2.csv',index = None)
sub['unit_sales'] = np.clip(np.expm1(sub['unit_sales']),0,900)
sub.to_csv('2lgb.csv', index = None)

In [249]:
submission2.head()

id  unit_sales
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040    0.191274
          99197    2017-08-16  125497041    0.401068
          103501   2017-08-16  125497042    0.000000
          103520   2017-08-16  125497043    1.422042
          103665   2017-08-16  125497044    1.995968

In [250]:
submission.head()

id  unit_sales
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040    0.203015
          99197    2017-08-16  125497041    0.384628
          103501   2017-08-16  125497042    0.000000
          103520   2017-08-16  125497043    1.268682
          103665   2017-08-16  125497044    1.921087

In [251]:
sub.head()

id  unit_sales
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040    0.197130
          99197    2017-08-16  125497041    0.392824
          103501   2017-08-16  125497042    0.000000
          103520   2017-08-16  125497043    1.344108
          103665   2017-08-16  125497044    1.958290